# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [15]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))

save (generic function with 1 method)

In [86]:
# TODOs
# - kmeans clustering
# - Fix weighting
# - Ramping <=== seems important...
# - Get correct costs
# - commit code


## EXTRA
# Quardatic costs



## Questions for Qi ##
# a) costs? How do you handel the fact that costs are quadratic? Also why do they include c0?
# b) 1e-8 * 100 * (2700000 / 20 + 12500) # What are the units of this? and why do batteries cost less than solar??


# 3 mins to solve 1 week of cap ex

In [88]:
#run_model(load, basic_capex, save, "san_diego_system", "test_senario")

In [160]:
(buses, lines, gens, loads, variability, P) = load("san_diego_system");

In [188]:
model = expansion_working(buses, lines, gens, loads, variability, P)

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 159552 rows, 68926 columns and 340916 nonzeros
Model fingerprint: 0xbc21bcea
Coefficient statistics:
  Matrix range     [1e-03, 1e+07]
  Objective range  [7e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-03, 3e+03]
Presolve removed 121770 rows and 35704 columns
Presolve time: 0.12s
Presolved: 37782 rows, 37422 columns, 137890 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.46s

Barrier statistics:
 Free vars  : 924
 AA' NZ     : 6.170e+05
 Factor NZ  : 5.231e+06 (roughly 70 MB of memory)
 Factor Ops : 3.565e+09 (less than 1 second per iteration)
 Threa

A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 68926
├ num_constraints: 187678
│ ├ AffExpr in MOI.EqualTo{Float64}: 44496
│ ├ AffExpr in MOI.GreaterThan{Float64}: 29088
│ ├ AffExpr in MOI.LessThan{Float64}: 85968
│ ├ VariableRef in MOI.EqualTo{Float64}: 24
│ └ VariableRef in MOI.GreaterThan{Float64}: 28102
└ Names registered in the model
  └ :CAP, :CHARGE, :FLOW, :GEN, :SHED, :SOC, :THETA, :cAngleLimitsMax, :cAngleLimitsMin, :cBalance, :cLineFlows, :cLineLimits, :cMaxCharge, :cMaxGenOld, :cMaxStateOfCharge, :cShed, :cStateOfCharge, :eFixedCostsSolar, :eFixedCostsStorage, :eNSECosts, :eTotalCosts, :eVariableCosts

In [190]:
save(model, "san_diego_system", "test_senario", buses, lines, gens, loads)

"/Users/adamsedlak/Documents/Julia/MAE243/final-project/models/san_diego_system/outputs/test_senario/costs.csv"

In [186]:
function expansion_working(buses, lines, gens, loads, variability, P)
    """
    Solve capacity expansion with CATS.
    
    Inputs:
        buses -- dataframe with bus types and loads
        lines -- dataframe with transmission lines info
        gens -- dataframe with generator info and costs
        loads -- 
        variability --
        P --
    """   
    #### HELPER FUNCTIONS ####
    # Select a line column using i and j 
    line = (i, j, col) -> lines[lines_map[(i, j)], col]
    # Subset of all lines connected to bus i
    J = i -> lines.t_bus[lines.f_bus .== i]
    # Select all generators in G connected to bus i
    sel = (G, i) -> collect(G)[gens[collect(G),:bus] .== i]

    function prev(t)
        """
        Returns the t-1 index using wrapping to ensure
        that if t-1 ∉ P then the largest t ∈ P is returned.
        """
        P_i = findfirst(P_i -> t in P_i, P)
        shift = first(P[P_i])
        p = length(P[P_i])
        return mod1(t - shift, p) + shift - 1
    end

    function get_CAP(g)
        """
        Return CAP decision variable if g ∈ G_new.
        Otherwise just return zero        
        """
        return g ∈ G_new ? CATS_Model[:CAP][g] : 0
    end
    
    ##### SETS ####
    G = gens[:, :gen_id]                 # G: Set of all generators
    G_new = gens[gens.canidate .==1, :gen_id]
    
    G_ess = gens[gens.ess .== 1, :gen_id]     # G_ess: Set of all energy storage system (ESS) generators
    G_solar = gens[gens.fueltype .== "Solar Photovoltaic", :gen_id]
    
    N = 1:nrow(buses)            # N: Set of all network nodes
    L = collect(zip(             # L: Set of all lines/branches
        lines.f_bus, lines.t_bus))

    T = 1:ncol(loads)            # T: Set of all hours

    #### GLOBALS ####
    # Mapping from (i, j) tuple to line index l
    lines_map = Dict(zip(L, 1:length(L)))
    
    #### PARAMETERS ####
    θlim = π*(60/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    
    storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
    η_charge = 0.97    # Charging efficiency  
    η_discharge = 0.91 # Discharging efficiecy
    
    # Toy parameters for a limited investment in solar at each bus
    shed_cost = 1000 
    budget = 0
    solar_cost = 1475
    battery_cost = 1475 

    ########################
    #####     MODEL    #####
    ########################
    
    CATS_Model = Model(() -> Gurobi.Optimizer())
    
    # Decision variables   
    @variables(CATS_Model, begin
        GEN[G,T] ≥ 0       # Generation of each generator 
        SHED[N,T] ≥ 0      # Load sheading at bus N
        THETA[N,T]         # Voltage phase angle of bus
        FLOW[L,T]          # Flows between all pairs of nodes
        SOC[G_ess,T] ≥ 0     # ESS state of charge
        CHARGE[G_ess,T] ≥ 0  # Chargeing of ESS 
        CAP[G_new] ≥ 0
    end)

    # Max and min generation constraints
    @constraint(CATS_Model, cMaxGenOld[g ∈ G, t ∈ T], 
        GEN[g,t] ≤ variability[g,t]*(gens[g,:pmax] + get_CAP(g)))

    # Load shedding constraint
    @constraint(CATS_Model, cShed[i ∈ N, t ∈ T], 
        SHED[i,t] ≤ loads[i,t])
    
    #########################
    #### BATTERY STORAGE ####
    #########################
    # Charging must be less than max power capacity
    @constraint(CATS_Model, cMaxCharge[g ∈ G_ess, t ∈ T], 
        CHARGE[g,t] ≤ gens[g,:pmax] + get_CAP(g))
    
    # Energy capacity must be less ×4 the power capacity
    @constraint(CATS_Model, cMaxStateOfCharge[g ∈ G_ess, t ∈ T], 
        SOC[g,t] ≤ storage_hrs*(gens[g,:pmax] + get_CAP(g)))
    
    # SOC in the next time is a function of SOC in the pervious time
    # with circular wrapping for the first and last t ∈ P_i
    @constraint(CATS_Model, cStateOfCharge[g ∈ G_ess, t ∈ T],
        SOC[g,t] == SOC[g,prev(t)] + (CHARGE[g,t]*η_charge - GEN[g,t]/η_charge))
    
    ########################
    ######## DC OPF ########
    ########################
    # Create slack bus with theta=0
    fix.(THETA[slack_bus, T], 0)
    
    # Max line flow must be less than the nominal (RATE A) of the line
    @constraint(CATS_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
        FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 
    
    # Angle limited to less than 60 degrees 
    @constraint(CATS_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≤  θlim)
    @constraint(CATS_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                    
    # Flow is governed by angle difference between buses
    @constraint(CATS_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
        FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

    # Power must be balanced at all nodes
    # Σ(generation) + shedding - Σ(demand) - Σ charging = Σ(flows) ∀ n ∈ N
    @constraint(CATS_Model, cBalance[i ∈ N, t ∈ T], 
        sum(GEN[g,t] for g ∈ sel(G, i))
        + SHED[i,t] - loads[i,t] 
        - sum(CHARGE[g,t] for g ∈ sel(G_ess, i))
         == sum(FLOW[(i,j),t] for j ∈ J(i)))

    #########################
    ####### OBJECTIVE #######
    #########################

    # The weighted operational costs of running each generator
    @expression(CATS_Model, eVariableCosts,
        sum(gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T))
    
    # The weighted operational costs of shedding load/non-served energy (NSE)
    @expression(CATS_Model, eNSECosts,
        sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T))

    # Fixed costs of all solar investments
	@expression(CATS_Model, eFixedCostsSolar,
        sum(solar_cost * CAP[g] for g ∈ setdiff(G_new, G_solar)))

    # Fixed costs of all battery/ESS investments
    @expression(CATS_Model, eFixedCostsStorage,
		sum(battery_cost * CAP[g] for g ∈ intersect(G_new, G_ess)))

    # Minimize the sum of investment and O&M costs 
	@expression(CATS_Model, eTotalCosts,
		eVariableCosts + eNSECosts + eFixedCostsStorage + eFixedCostsSolar)
    
	@objective(CATS_Model, Min, eTotalCosts)

    optimize!(CATS_Model)

    return CATS_Model
end

expansion_working (generic function with 1 method)

In [130]:
maximum(value.(model[:CAP]))

76.94450549450548

true

In [180]:
model[:cStateOfCharge][4, :]

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:24
And data, a 23-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 cStateOfCharge[4,2] : 1.0309278350515465 GEN[4,2] - SOC[4,1] + SOC[4,2] - 0.97 CHARGE[4,2] = 0
 cStateOfCharge[4,3] : 1.0309278350515465 GEN[4,3] - SOC[4,2] + SOC[4,3] - 0.97 CHARGE[4,3] = 0
 cStateOfCharge[4,4] : 1.0309278350515465 GEN[4,4] - SOC[4,3] + SOC[4,4] - 0.97 CHARGE[4,4] = 0
 cStateOfCharge[4,5] : 1.0309278350515465 GEN[4,5] - SOC[4,4] + SOC[4,5] - 0.97 CHARGE[4,5] = 0
 cStateOfCharge[4,6] : 1.0309278350515465 GEN[4,6] - SOC[4,5] + SOC[4,6] - 0.97 CHARGE[4,6] = 0
 cStateOfCharge[4,7] : 1.0309278350515465 GEN[4,7] - SOC[4,6] + SOC[4,7] - 0.97 CHARGE[4,7] = 0
 cStateOfCharge

In [148]:
df[df[:, :CAP] .!= 0, :]

Row,bus,fueltype,c2,c1,c0,pmax,ess,canidate,gen_id,CAP
,Int64,String,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64
1,14,Solar Photovoltaic,0.0,0.0,0.0,0.0,0,1,87,76.9445
2,36,Solar Photovoltaic,0.0,0.0,0.0,0.0,0,1,209,23.0555


In [182]:

#### HELPER FUNCTIONS ####
# Select a line column using i and j 
line = (i, j, col) -> lines[lines_map[(i, j)], col]
# Subset of all lines connected to bus i
J = i -> lines.t_bus[lines.f_bus .== i]
# Select all generators in G connected to bus i
sel = (G, i) -> collect(G)[gens[collect(G),:bus] .== i]

function prev(t)
    """
    Returns the t-1 index using wrapping to ensure
    that if t-1 ∉ P then the largest t ∈ P is returned.
    """
    P_i = findfirst(P_i -> t in P_i, P)
    shift = first(P[P_i])
    p = length(P[P_i])
    return mod1(t - shift, p) + shift - 1
end

function get_CAP(g)
    """
    Return CAP decision variable if g ∈ G_new.
    Otherwise just return zero        
    """
    return g ∈ G_new ? CATS_Model[:CAP][g] : 0
end

##### SETS ####
G = gens[:, :gen_id]                 # G: Set of all generators
G_new = gens[gens.canidate .==1, :gen_id]

G_ess = gens[gens.ess .== 1, :gen_id]     # G_ess: Set of all energy storage system (ESS) generators
G_solar = gens[gens.fueltype .== "Solar Photovoltaic", :gen_id]

N = 1:nrow(buses)            # N: Set of all network nodes
L = collect(zip(             # L: Set of all lines/branches
    lines.f_bus, lines.t_bus))

T = 1:ncol(loads)            # T: Set of all hours

#### GLOBALS ####
# Mapping from (i, j) tuple to line index l
lines_map = Dict(zip(L, 1:length(L)))

#### PARAMETERS ####
θlim = π*(60/180)  # Absolute max angle limit in rad
slack_bus = 416    # Set slack bus to one with many CTs

storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
η_charge = 0.97    # Charging efficiency  
η_discharge = 0.91 # Discharging efficiecy

# Toy parameters for a limited investment in solar at each bus
shed_cost = 1000 
budget = 0
solar_cost = 1475
battery_cost = 1475 

########################
#####     MODEL    #####
########################

CATS_Model = Model(() -> Gurobi.Optimizer())

# Decision variables   
@variables(CATS_Model, begin
    GEN[G,T] ≥ 0       # Generation of each generator 
    SHED[N,T] ≥ 0      # Load sheading at bus N
    THETA[N,T]         # Voltage phase angle of bus
    FLOW[L,T]          # Flows between all pairs of nodes
    SOC[G_ess,T] ≥ 0     # ESS state of charge
    CHARGE[G_ess,T] ≥ 0  # Chargeing of ESS 
    CAP[G_new] ≥ 0
end)

# Max and min generation constraints
@constraint(CATS_Model, cMaxGenOld[g ∈ G, t ∈ T], 
    GEN[g,t] ≤ variability[g,t]*(gens[g,:pmax] + get_CAP(g)))

# Load shedding constraint
@constraint(CATS_Model, cShed[i ∈ N, t ∈ T], 
    SHED[i,t] ≤ loads[i,t])

#########################
#### BATTERY STORAGE ####
#########################
# Charging must be less than max power capacity
@constraint(CATS_Model, cMaxCharge[g ∈ G_ess, t ∈ T], 
    CHARGE[g,t] ≤ gens[g,:pmax] + get_CAP(g))

# Energy capacity must be less ×4 the power capacity
@constraint(CATS_Model, cMaxStateOfCharge[g ∈ G_ess, t ∈ T], 
    SOC[g,t] ≤ storage_hrs*(gens[g,:pmax] + get_CAP(g)))

# SOC in the next time is a function of SOC in the pervious time
# with circular wrapping for the first and last t ∈ P_i
@constraint(CATS_Model, cStateOfCharge[g ∈ G_ess, t ∈ T[2:24]],
    SOC[g,t] == SOC[g,prev(t)] + (CHARGE[g,t]*η_charge - GEN[g,t]/η_charge))

########################
######## DC OPF ########
########################
# Create slack bus with theta=0
fix.(THETA[slack_bus, T], 0)

# Max line flow must be less than the nominal (RATE A) of the line
@constraint(CATS_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
    FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 

# Angle limited to less than 60 degrees 
@constraint(CATS_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
    (THETA[i,t] - THETA[j,t]) ≤  θlim)
@constraint(CATS_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
    (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                
# Flow is governed by angle difference between buses
@constraint(CATS_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
    FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

# Power must be balanced at all nodes
# Σ(generation) + shedding - Σ(demand) - Σ charging = Σ(flows) ∀ n ∈ N
@constraint(CATS_Model, cBalance[i ∈ N, t ∈ T], 
    sum(GEN[g,t] for g ∈ sel(G, i))
    + SHED[i,t] - loads[i,t] 
    - sum(CHARGE[g,t] for g ∈ sel(G_ess, i))
     == sum(FLOW[(i,j),t] for j ∈ J(i)))

#########################
####### OBJECTIVE #######
#########################

# The weighted operational costs of running each generator
@expression(CATS_Model, eVariableCosts,
    sum(gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T))

# The weighted operational costs of shedding load/non-served energy (NSE)
@expression(CATS_Model, eNSECosts,
    sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T))

# Fixed costs of all solar investments
@expression(CATS_Model, eFixedCostsSolar,
    sum(solar_cost * CAP[g] for g ∈ setdiff(G_new, G_solar)))

# Fixed costs of all battery/ESS investments
@expression(CATS_Model, eFixedCostsStorage,
    sum(battery_cost * CAP[g] for g ∈ intersect(G_new, G_ess)))

# Minimize the sum of investment and O&M costs 
@expression(CATS_Model, eTotalCosts,
    eVariableCosts + eNSECosts + eFixedCostsStorage + eFixedCostsSolar)

@objective(CATS_Model, Min, eTotalCosts)

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22


31.3951746 GEN[1,1] + 31.3951746 GEN[1,2] + 31.3951746 GEN[1,3] + 31.3951746 GEN[1,4] + 31.3951746 GEN[1,5] + 31.3951746 GEN[1,6] + 31.3951746 GEN[1,7] + 31.3951746 GEN[1,8] + 31.3951746 GEN[1,9] + 31.3951746 GEN[1,10] + 31.3951746 GEN[1,11] + 31.3951746 GEN[1,12] + 31.3951746 GEN[1,13] + 31.3951746 GEN[1,14] + 31.3951746 GEN[1,15] + 31.3951746 GEN[1,16] + 31.3951746 GEN[1,17] + 31.3951746 GEN[1,18] + 31.3951746 GEN[1,19] + 31.3951746 GEN[1,20] + 31.3951746 GEN[1,21] + 31.3951746 GEN[1,22] + 31.3951746 GEN[1,23] + 31.3951746 GEN[1,24] + 14.8788144 GEN[2,1] + 14.8788144 GEN[2,2] + 14.8788144 GEN[2,3] + 14.8788144 GEN[2,4] + 14.8788144 GEN[2,5] + 14.8788144 GEN[2,6] + [[...16700 terms omitted...]] + 2950 CAP[334] + 2950 CAP[335] + 2950 CAP[336] + 2950 CAP[337] + 2950 CAP[338] + 2950 CAP[339] + 2950 CAP[340] + 2950 CAP[341] + 2950 CAP[342] + 2950 CAP[343] + 2950 CAP[344] + 2950 CAP[345] + 2950 CAP[346] + 2950 CAP[347] + 2950 CAP[348] + 2950 CAP[349] + 2950 CAP[350] + 2950 CAP[351] + 2950 

In [184]:
T

1:24